<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/7_Finans_i%C3%A7in_Python_EMA_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Kütüphanelerin yüklenmesi
!pip install git+https://github.com/rongardF/tvdatafeed
!pip install tradingview-screener
!pip install backtesting


#kütüphanelerin çağırılması
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from backtesting import Backtest, Strategy

#Tradingview Kütüphanesini açıyoruz.
tv = TvDatafeed()

#EMA hesaplamasını tanımlıyoruz.
def ema(series, length):
    calc = series.ewm(span=length, adjust=False).mean()
    return calc

#Her Zaman kullanacağımız Stratejimizi hazırlıyoruz.
class Strategy(Strategy):
    def init(self):
        pass
    def next(self):
        if self.data['Entry'] == True and not self.position:
            self.buy()
        elif self.data['Exit'] == True:
            self.position.close()

#Hisse Adlarını Tanımlıyoruz.
Hisse = ['AKBNK','ALARK','ASELS','ASTOR','BIMAS',
	'BRSAN','DOAS','EKGYO','ENKAI','EREGL',
	'FROTO','GARAN','GUBRF','HEKTS','ISCTR',
	'KCHOL','KONTR','KOZAL','KRDMD','OYAKC',
	'PETKM','PGSUS','SAHOL','SASA','SISE',
	'TCELL','THYAO','TOASO','TUPRS','YKBNK']

# BIST30 sonuçları Arşivlemek için İki Adet Boş dataframe yapısı oluşturuyoruz.
df_bist30_RE = pd.DataFrame(columns=['Hisse Adı','EMA Kısa', 'EMA Uzun','Getiri Yüzdesi'])
df_bist30_WR = pd.DataFrame(columns=['Hisse Adı','EMA Kısa', 'EMA Uzun','Kazanma Oranı'])

for z in range(len(Hisse)):
    #Hisse Verilerini Tradingview Kütüphanesinden çekiyoruz.
    print(Hisse[z])
    data = tv.get_hist(symbol=Hisse[z], exchange='BIST', interval=Interval.in_daily, n_bars=1000)
    #data başlıklarını yeniden isimlendiriyoruz
    data.rename(columns={'open': 'Open', 'high': 'High',
                          'low': 'Low',
                          'close': 'Close',
                          'volume': 'Volume'},
                          inplace=True)

    #Sonuçları Arşivlemek için İki Adet Boş dataframe yapısı oluşturuyoruz.
    df_RE = pd.DataFrame(columns=['EMA Kısa', 'EMA Uzun','Getiri Yüzdesi'])
    df_WR = pd.DataFrame(columns=['EMA Kısa', 'EMA Uzun','Kazanma Oranı'])

    for i in range(5, 51):
        data['EMA_L1'] = ema(data['Close'], i)
        for j in range(i+1, 51):
            data['EMA_L2'] = ema(data['Close'], j)
            data['Entry'] = data['EMA_L1'] > data['EMA_L2']
            data['Exit'] = data['EMA_L1'] < data['EMA_L2']

            # Backtest yapımızı çağırıyoruz ve sonuçları inceliyoruz.
            bt = Backtest(data, Strategy, cash=100000, commission=0.002)
            Stats = bt.run()
            RE = round(Stats.loc['Return [%]'], 2)
            WR = round(Stats.loc['Win Rate [%]'], 2)

            df_RE.loc[len(df_RE)] = [i,j,RE]
            df_WR.loc[len(df_WR)] = [i,j,WR]

    #Sonuçları yazdırma.
    df_RE = df_RE.sort_values(by='Getiri Yüzdesi', ascending=False)
    df_WR = df_WR.sort_values(by='Kazanma Oranı', ascending=False)
    RE_list = df_RE.iloc[0].tolist()
    WR_list = df_WR.iloc[0].tolist()
    df_bist30_RE.loc[len(df_bist30_RE)] = [Hisse[z]]+RE_list
    df_bist30_WR.loc[len(df_bist30_WR)] = [Hisse[z]]+WR_list

print(df_bist30_RE)
print(df_bist30_WR)

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-soltfi5i
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-soltfi5i
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=b5f2fe166f06817f7bb6e24470aac42b341c98c5928dbc106f995611a9926365
  Stored in directory: /tmp/pip-ephem-wheel-cache-ufwsenn2/wheels/e4/32/1e/21ebcacc6549d75fae3bf3ff75cee8fcbe4e5c189d88fbd5a1
Successfully built tvdatafeed
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 764.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 48.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency res

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


AKBNK
ALARK
ASELS
ASTOR
BIMAS
BRSAN
DOAS
EKGYO
ENKAI
EREGL
FROTO
GARAN
GUBRF
HEKTS
ISCTR
KCHOL
KONTR
KOZAL
KRDMD
OYAKC
PETKM
PGSUS
SAHOL
SASA
SISE
TCELL
THYAO
TOASO
TUPRS
YKBNK
   Hisse Adı  EMA Kısa  EMA Uzun  Getiri Yüzdesi
0      AKBNK      42.0      50.0          829.60
1      ALARK      11.0      27.0         3193.52
2      ASELS      30.0      49.0          483.49
3      ASTOR      13.0      16.0          794.45
4      BIMAS      49.0      50.0          544.90
5      BRSAN      19.0      41.0         4020.09
6       DOAS      14.0      36.0         1847.58
7      EKGYO       5.0       6.0          517.56
8      ENKAI      10.0      16.0          384.15
9      EREGL       7.0      19.0          424.14
10     FROTO      49.0      50.0          808.28
11     GARAN      46.0      48.0         1249.34
12     GUBRF       9.0      48.0          769.43
13     HEKTS      38.0      39.0         1655.73
14     ISCTR      37.0      41.0         1412.97
15     KCHOL      34.0      46.0       